In [ ]:
import pandas as pd
import requests

# Your 76-city list
cities_list = [
    "Aberdeen", "Armagh", "Bangor", "Bath", "Belfast", "Birmingham", "Bradford",
    "Brighton and Hove", "Bristol", "Cambridge", "Cardiff", "Carlisle", "Chelmsford",
    "Chester", "Chichester", "City of London", "City of Westminster", "Colchester",
    "Coventry", "Derby", "Derry", "Doncaster", "Dundee", "Dunfermline", "Durham",
    "Edinburgh", "Ely", "Exeter", "Glasgow", "Gloucester", "Hereford", "Inverness",
    "Kingston upon Hull", "Lancaster", "Leeds", "Leicester", "Lichfield", "Lincoln",
    "Lisburn", "Liverpool", "Manchester", "Milton Keynes", "Newcastle upon Tyne",
    "Newport", "Newry", "Norwich", "Nottingham", "Oxford", "Perth", "Peterborough",
    "Plymouth", "Portsmouth", "Preston", "Ripon", "Salford", "Salisbury", "Sheffield",
    "Southampton", "Southend-on-Sea", "St Albans", "St Asaph", "St Davids", "Stirling",
    "Stoke-on-Trent", "Sunderland", "Swansea", "Truro", "Wakefield", "Wells", "Winchester",
    "Wolverhampton", "Worcester", "Wrexham", "York"
]

# Step 1: fetch page manually
url = "http://lovemytown.co.uk/populations/townstable1.asp"
response = requests.get(url)
response.encoding = "latin1"   # or 'cp1252'
html = response.text

# Step 2: parse tables, explicitly specifying encoding
# Corrected: explicitly use 'latin1' encoding and select the second table (index 1)
tables = pd.read_html(html, encoding='latin1')
print(f"Found {len(tables)} tables on the page")

# Inspect table 0 and 1 to see structure
for idx, tbl in enumerate(tables[:2]):  # only first 2 to reduce output
    print("Table", idx, "columns:", tbl.columns.tolist())
    print(tbl.head(3))

# Step 3: select the correct table (likely 0 or 1)
# Corrected: Use the second table (index 1) as it contains the desired data
city_tbl = tables[1]   # adjust if needed

# Normalize city column name
# The column with city names is the second column (index 1) in table 1
city_tbl = city_tbl.rename(columns={city_tbl.columns[1]: "city"})


# Step 4: filter your 76 cities
filtered = city_tbl[city_tbl["city"].isin(cities_list)].copy()

print("Filtered subset:")
print(filtered)

# Save CSV
filtered.to_csv("selected_uk_cities_basic_info.csv", index=False, encoding="utf-8")

/tmp/ipython-input-338592424.py:28: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, encoding='latin1')


Found 2 tables on the page
Table 0 columns: [0, 1, 2]
    0   1   2
0 NaN NaN NaN
1 NaN NaN NaN
2 NaN NaN NaN
Table 1 columns: [0, 1, 2, 3]
     0           1           2       3
0  NaN        Town  Population  Status
1  1.0      London     8173941    City
2  2.0  Birmingham     1085810    City
Filtered subset:
         0        city        2     3
2      2.0  Birmingham  1085810  City
3      3.0     Glasgow   590507  City
4      4.0   Liverpool   552267  City
5      5.0     Bristol   535907  City
6      6.0   Sheffield   518090  City
..     ...         ...      ...   ...
506  506.0       Truro    20332  City
543  543.0         Ely    19090  City
575  575.0      Bangor    17988  City
620  620.0       Ripon    16363  City
877  877.0       Wells    10536  City

[63 rows x 4 columns]


In [ ]:
!pip install openaq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 2.0 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd

# Your WAQI API token
API_TOKEN = 'a0f1891a3b933228f68e9791a7796565ec701db8'

# Base URL for WAQI API
BASE_URL = 'https://api.waqi.info/feed/'

cities = ['Aberdeen', 'Armagh', 'Bangor, northern ireland', 'Bangor, wales', 'Bath, uk', 'Belfast', 'Birmingham', 'Bradford', 'Brighton and Hove', 'Bristol', 'Cambridge', 'Canterbury', 'Cardiff', 'Carlisle', 'Chelmsford', 'Chester', 'Chichester', 'City of London', 'City of Westminster', 'Colchester', 'Coventry', 'Derby', 'Derry', 'Doncaster', 'Dundee', 'Dunfermline', 'Durham', 'Edinburgh', 'Ely', 'Exeter', 'Glasgow', 'Gloucester', 'Hereford', 'Inverness', 'Kingston upon Hull', 'Lancaster', 'Leeds', 'Leicester', 'Lichfield', 'Lincoln', 'Lisburn', 'Liverpool', 'Manchester', 'Milton Keynes', 'Newcastle', 'Newport', 'Newry', 'Norwich', 'Nottingham', 'Oxford', 'Perth', 'Peterborough', 'Plymouth', 'Portsmouth', 'Preston', 'Ripon', 'Salford', 'Salisbury', 'Sheffield', 'Southampton', 'Southend-on-Sea', 'St Albans', 'St Asaph', 'St Davids', 'Stirling', 'Stoke-on-Trent', 'Sunderland', 'Swansea', 'Truro', 'Wakefield', 'Wells', 'Winchester', 'Wolverhampton', 'Worcester', 'Wrexham', 'York']

# Initialize an empty list to store results
data = []

# Fetch data for each city
for city in cities:
    url = f"{BASE_URL}{city}/?token={API_TOKEN}"
    response = requests.get(url)
    if response.status_code == 200:
        result = response.json()
        if result['status'] == 'ok':
            data.append({
                'City': city,
                'AQI': result['data']['aqi'],
                'Dominant Pollutant': result['data']['dominentpol'],
                'Time': result['data']['time']['s']
            })

# Convert to DataFrame
df_waqi = pd.DataFrame(data)

# Display the data
df_waqi.to_csv("air_quality.csv", index=False)
print(df_waqi)


                   City  AQI Dominant Pollutant                 Time
0              Aberdeen   27               pm25  2025-10-04 19:00:00
1               Belfast    9               pm10  2025-10-04 19:00:00
2            Birmingham   13               pm25  2021-10-01 15:00:00
3               Bristol   30               pm25  2025-10-04 19:00:00
4             Cambridge   30               pm25  2022-05-13 09:00:00
5               Cardiff   24                 o3  2025-10-04 20:00:00
6              Carlisle   46               pm25  2021-06-21 11:00:00
7            Chelmsford    2                 co  2019-01-03 04:00:00
8               Chester   43                 o3  2025-10-04 14:00:00
9        City of London   38               pm25  2025-10-04 19:00:00
10  City of Westminster   38               pm25  2025-10-04 19:00:00
11           Colchester    -                     2024-02-04 19:00:00
12             Coventry   25                 o3  2025-10-04 19:00:00
13                Derby  195      

In [ ]:
pip install overpy

In [ ]:
import overpy

# Initialize API
api = overpy.Overpass()

# Query clinics in Leeds
query = """
area["name"="Chelmsford"]["boundary"="administrative"]->.searchArea;
(
  node["amenity"="clinic"](area.searchArea);
  way["amenity"="clinic"](area.searchArea);
  relation["amenity"="clinic"](area.searchArea);
);
out center;
"""

result = api.query(query)

# Count the number of clinics
num_clinics = len(result.nodes) + len(result.ways) + len(result.relations)
print(f"Number of clinics in Leeds: {num_clinics}")

Number of clinics in Leeds: 15


In [ ]:
import overpy
import pandas as pd
import time

# Initialize Overpass API
api = overpy.Overpass()

# List of 76 UK cities
cities = [
    "Aberdeen", "Armagh", "Bangor (NI)", "Bangor (W)", "Bath", "Belfast", "Birmingham",
    "Bradford", "Brighton and Hove", "Bristol", "Cambridge", "Canterbury", "Cardiff",
    "Carlisle", "Chelmsford", "Chester", "Chichester", "City of London",
    "City of Westminster", "Colchester", "Coventry", "Derby", "Derry", "Doncaster",
    "Dundee", "Dunfermline", "Durham", "Edinburgh", "Ely", "Exeter", "Glasgow",
    "Gloucester", "Hereford", "Inverness", "Kingston upon Hull", "Lancaster", "Leeds",
    "Leicester", "Lichfield", "Lincoln", "Lisburn", "Liverpool", "Manchester",
    "Milton Keynes", "Newcastle upon Tyne", "Newport", "Newry", "Norwich", "Nottingham",
    "Oxford", "Perth", "Peterborough", "Plymouth", "Portsmouth", "Preston", "Ripon",
    "Salford", "Salisbury", "Sheffield", "Southampton", "Southend-on-Sea", "St Albans",
    "St Asaph", "St Davids", "Stirling", "Stoke-on-Trent", "Sunderland", "Swansea",
    "Truro", "Wakefield", "Wells", "Winchester", "Wolverhampton", "Worcester",
    "Wrexham", "York"
]

# Initialize a list to store results
data = []

# Function to query Overpass for given amenity
def get_amenity_count(city_name, amenity):
    query = f"""
    area["name"="{city_name}"]["boundary"="administrative"]->.searchArea;
    (
      node["amenity"="{amenity}"](area.searchArea);
      way["amenity"="{amenity}"](area.searchArea);
      relation["amenity"="{amenity}"](area.searchArea);
    );
    out center;
    """
    try:
        result = api.query(query)
        return len(result.nodes) + len(result.ways) + len(result.relations)
    except Exception as e:
        print(f"Error fetching {amenity} for {city_name}: {e}")
        return None

# Loop through all cities
for city in cities:
    print(f"Processing {city}...")
    hospitals = get_amenity_count(city, "hospital")
    university = get_amenity_count(city, "university")

    data.append({
        "City": city,
        "University": university,
    })

    # Sleep 1 second to avoid overloading the API
    time.sleep(3)

# Convert to DataFrame and save as CSV
df = pd.DataFrame(data)
df.to_csv("uk_uni.csv", index=False)
print("Saved healthcare data to 'uni.csv'")


Processing Aberdeen...
Processing Armagh...
Processing Bangor (NI)...
Processing Bangor (W)...
Processing Bath...
Processing Belfast...
Processing Birmingham...
Processing Bradford...
Processing Brighton and Hove...
Processing Bristol...
Processing Cambridge...
Processing Canterbury...
Processing Cardiff...
Processing Carlisle...
Processing Chelmsford...
Processing Chester...
Processing Chichester...
Processing City of London...
Processing City of Westminster...
Processing Colchester...
Processing Coventry...
Processing Derby...
Processing Derry...
Processing Doncaster...
Processing Dundee...
Processing Dunfermline...
Processing Durham...
Processing Edinburgh...
Processing Ely...
Processing Exeter...
Processing Glasgow...
Processing Gloucester...
Processing Hereford...
Processing Inverness...
Processing Kingston upon Hull...
Processing Lancaster...
Processing Leeds...
Processing Leicester...
Processing Lichfield...
Processing Lincoln...
Processing Lisburn...
Processing Liverpool...
Pro

In [ ]:
import requests
import pandas as pd
import time
from collections import Counter

cities = [
    {"name": "Aberdeen", "lat": 57.1498891, "lon": -2.0937528},
    {"name": "Armagh", "lat": 54.3502798, "lon": -6.652792},
    {"name": "Bangor(NI)", "lat": 54.6538, "lon": -5.6736},
    {"name": "Bangor(W)", "lat": 53.22739, "lon": -4.129263},
    {"name": "Bath", "lat": 51.3781018, "lon": -2.3596827},
    {"name": "Belfast", "lat": 54.597285, "lon": -5.93012},
    {"name": "Birmingham", "lat": 52.4822694, "lon": -1.8900078},
    {"name": "Bradford", "lat": 53.7937996, "lon": -1.7563583},
    {"name": "Brighton and Hove", "lat": 50.8351605, "lon": -0.1261028},
    {"name": "Bristol", "lat": 51.454513, "lon": -2.58791},
    {"name": "Cambridge", "lat": 52.1950788, "lon": 0.1312729},
    {"name": "Canterbury", "lat": 51.2770331, "lon": 1.0837855},
    {"name": "Cardiff", "lat": 51.483707, "lon": -3.1680962},
    {"name": "Carlisle", "lat": 54.892473, "lon": -2.932931},
    {"name": "Chelmsford", "lat": 51.7355868, "lon": 0.4685497},
    {"name": "Chester", "lat": 53.193392, "lon": -2.893075},
    {"name": "Chichester", "lat": 50.83761, "lon": -0.774936},
    {"name": "City of London", "lat": 51.5134068, "lon": -0.0890441},
    {"name": "City of Westminster", "lat": 51.5072052, "lon": -0.1276514},
    {"name": "Colchester", "lat": 51.895927, "lon": 0.891874},
    {"name": "Coventry", "lat": 52.4128163, "lon": -1.5089521},
    {"name": "Derby", "lat": 52.9225301, "lon": -1.4746186},
    {"name": "Derry", "lat": 54.9964705, "lon": -7.3101358},
    {"name": "Doncaster", "lat": 53.5227501, "lon": -1.132336},
    {"name": "Dundee", "lat": 56.462018, "lon": -2.970721},
    {"name": "Dunfermline", "lat": 56.071741, "lon": -3.452151},
    {"name": "Durham", "lat": 54.7753, "lon": -1.5849},
    {"name": "Edinburgh", "lat": 55.953252, "lon": -3.188267},
    {"name": "Ely", "lat": 52.399539, "lon": 0.262363},
    {"name": "Exeter", "lat": 50.7260367, "lon": -3.5274889},
    {"name": "Glasgow", "lat": 55.8616704, "lon": -4.2583345},
    {"name": "Gloucester", "lat": 51.8642449, "lon": -2.238156},
    {"name": "Hereford", "lat": 52.056398, "lon": -2.715974},
    {"name": "Inverness", "lat": 57.477773, "lon": -4.224721},
    {"name": "Kingston upon Hull", "lat": 53.7674909, "lon": -0.3273025},
    {"name": "Lancaster", "lat": 54.0449072, "lon": -2.7993467},
    {"name": "Leeds", "lat": 53.8007554, "lon": -1.5490774},
    {"name": "Leicester", "lat": 52.6368778, "lon": -1.1397592},
    {"name": "Lichfield", "lat": 52.681602, "lon": -1.831672},
    {"name": "Lincoln", "lat": 53.2331, "lon": -0.5392},
    {"name": "Lisburn", "lat": 54.516246, "lon": -6.0580106},
    {"name": "Liverpool", "lat": 53.4083714, "lon": -2.9915726},
    {"name": "Manchester", "lat": 53.4807593, "lon": -2.2426305},
    {"name": "Milton Keynes", "lat": 52.0406224, "lon": -0.7594171},
    {"name": "Newcastle", "lat": 54.978252, "lon": -1.61778},
    {"name": "Newport", "lat": 51.5842, "lon": -2.9977},
    {"name": "Newry", "lat": 54.1751024, "lon": -6.3402299},
    {"name": "Norwich", "lat": 52.6292567, "lon": 1.2978802},
    {"name": "Nottingham", "lat": 52.9540223, "lon": -1.1549892},
    {"name": "Oxford", "lat": 51.7520209, "lon": -1.2577263},
    {"name": "Perth", "lat": 56.3969, "lon": -3.4370},
    {"name": "Peterborough", "lat": 52.5703169, "lon": -0.2407997},
    {"name": "Plymouth", "lat": 50.3754565, "lon": -4.1426565},
    {"name": "Portsmouth", "lat": 50.8197675, "lon": -1.0879769},
    {"name": "Preston", "lat": 53.7591576, "lon": -2.7046445},
    {"name": "Ripon", "lat": 54.1361346, "lon": -1.5237756},
    {"name": "Salford", "lat": 53.4872927, "lon": -2.2900071},
    {"name": "Salisbury", "lat": 51.068785, "lon": -1.794472},
    {"name": "Sheffield", "lat": 53.381129, "lon": -1.470085},
    {"name": "Southampton", "lat": 50.9105468, "lon": -1.4049018},
    {"name": "Southend-on-Sea", "lat": 51.5459269, "lon": 0.7077123},
    {"name": "St Albans", "lat": 51.7508784, "lon": -0.3397723},
    {"name": "St Asaph", "lat": 53.2577125, "lon": -3.4418762},
    {"name": "St Davids", "lat": 51.881227, "lon": -5.265995},
    {"name": "Stirling", "lat": 56.1165227, "lon": -3.9369029},
    {"name": "Stoke-on-Trent", "lat": 53.0033369, "lon": -2.1827408},
    {"name": "Sunderland", "lat": 54.9044161, "lon": -1.3811746},
    {"name": "Swansea", "lat": 51.62144, "lon": -3.943646},
    {"name": "Truro", "lat": 50.263195, "lon": -5.051041},
    {"name": "Wakefield", "lat": 53.6848162, "lon": -1.5038596},
    {"name": "Wells", "lat": 51.2104862, "lon": -2.6467584},
    {"name": "Winchester", "lat": 51.059771, "lon": -1.310142},
    {"name": "Wolverhampton", "lat": 52.5868159, "lon": -2.1256587},
    {"name": "Worcester", "lat": 52.1941389, "lon": -2.2190523},
    {"name": "Wrexham", "lat": 53.04304, "lon": -2.992494},
    {"name": "York", "lat": 53.9614205, "lon": -1.0739108}
]

data = []

for city in cities:
    #print(f"Fetching crime data for {city['name']}...")

    url = f"https://data.police.uk/api/crimes-at-location?date=2024-01&lat={city['lat']}&lng={city['lon']}"
    try:
        response = requests.get(url)
        crimes = response.json()
        total_crimes = len(crimes)

        # Count number of each type of crime
        crime_types = [crime['category'] for crime in crimes]
        crime_count_by_type = Counter(crime_types)

        # Build a row for this city
        row = {
            "City": city['name'],
            "Total_Crimes": total_crimes,
        }
        # Add each crime type as a separate column
        for ctype, count in crime_count_by_type.items():
            row[ctype] = count
    except Exception as e:
        print(f"Error fetching data for {city['name']}: {e}")
        row = {"City": city['name'], "Total_Crimes": None}

    data.append(row)

    time.sleep(3)  # avoid hitting rate limits

# Convert to DataFrame
df = pd.DataFrame(data)

# Save to CSV
print(df)
#df.to_csv("uk_crime.csv", index=False)

             City  Total_Crimes  criminal-damage-arson  drugs  other-theft  \
0        Aberdeen             3                    1.0    1.0          1.0   
1          Armagh             1                    NaN    NaN          NaN   
2      Bangor(NI)             0                    NaN    NaN          NaN   
3       Bangor(W)             4                    1.0    NaN          1.0   
4            Bath             5                    NaN    NaN          NaN   
..            ...           ...                    ...    ...          ...   
71     Winchester             0                    NaN    NaN          NaN   
72  Wolverhampton             7                    1.0    NaN          NaN   
73      Worcester             3                    NaN    NaN          1.0   
74        Wrexham             0                    NaN    NaN          NaN   
75           York             1                    NaN    NaN          NaN   

    shoplifting  violent-crime  bicycle-theft  anti-social-beha

In [ ]:
import requests
from collections import Counter
import pandas as pd
import time

# Example: Leeds grid points (lat/lon) to cover city
leeds_points = [
    (54.6538, -5.6736)
]

total_crimes = Counter()

for lat, lon in leeds_points:
    url = f"https://data.police.uk/api/crimes-at-location?date=2024-01&lat={lat}&lng={lon}"
    resp = requests.get(url)
    crimes = resp.json()
    types = [c['category'] for c in crimes]
    total_crimes.update(types)
    time.sleep(1)

print("Total crimes in Leeds:", sum(total_crimes.values()))
print("Breakdown by type:", dict(total_crimes))

Total crimes in Leeds: 0
Breakdown by type: {}


In [ ]:
import requests
import pandas as pd
import time
from collections import Counter

# List of months to query (you can change the year)
months = [f"2024-{m:02d}" for m in range(1, 13)]

# --- List of cities (same as yours shortened for example, use full list) ---
cities = [
    {"name": "Aberdeen", "lat": 57.1498891, "lon": -2.0937528},
    {"name": "Armagh", "lat": 54.3502798, "lon": -6.652792},
    {"name": "Bangor(NI)", "lat": 54.6538, "lon": -5.6736},
    {"name": "Bangor(W)", "lat": 53.22739, "lon": -4.129263},
    {"name": "Bath", "lat": 51.3781018, "lon": -2.3596827},
    {"name": "Belfast", "lat": 54.597285, "lon": -5.93012},
    {"name": "Birmingham", "lat": 52.4822694, "lon": -1.8900078},
    {"name": "Bradford", "lat": 53.7937996, "lon": -1.7563583},
    {"name": "Brighton and Hove", "lat": 50.8351605, "lon": -0.1261028},
    {"name": "Bristol", "lat": 51.454513, "lon": -2.58791},
    {"name": "Cambridge", "lat": 52.1950788, "lon": 0.1312729},
    {"name": "Canterbury", "lat": 51.2770331, "lon": 1.0837855},
    {"name": "Cardiff", "lat": 51.483707, "lon": -3.1680962},
    {"name": "Carlisle", "lat": 54.892473, "lon": -2.932931},
    {"name": "Chelmsford", "lat": 51.7355868, "lon": 0.4685497},
    {"name": "Chester", "lat": 53.193392, "lon": -2.893075},
    {"name": "Chichester", "lat": 50.83761, "lon": -0.774936},
    {"name": "City of London", "lat": 51.5134068, "lon": -0.0890441},
    {"name": "City of Westminster", "lat": 51.5072052, "lon": -0.1276514},
    {"name": "Colchester", "lat": 51.895927, "lon": 0.891874},
    {"name": "Coventry", "lat": 52.4128163, "lon": -1.5089521},
    {"name": "Derby", "lat": 52.9225301, "lon": -1.4746186},
    {"name": "Derry", "lat": 54.9964705, "lon": -7.3101358},
    {"name": "Doncaster", "lat": 53.5227501, "lon": -1.132336},
    {"name": "Dundee", "lat": 56.462018, "lon": -2.970721},
    {"name": "Dunfermline", "lat": 56.071741, "lon": -3.452151},
    {"name": "Durham", "lat": 54.7753, "lon": -1.5849},
    {"name": "Edinburgh", "lat": 55.953252, "lon": -3.188267},
    {"name": "Ely", "lat": 52.399539, "lon": 0.262363},
    {"name": "Exeter", "lat": 50.7260367, "lon": -3.5274889},
    {"name": "Glasgow", "lat": 55.8616704, "lon": -4.2583345},
    {"name": "Gloucester", "lat": 51.8642449, "lon": -2.238156},
    {"name": "Hereford", "lat": 52.056398, "lon": -2.715974},
    {"name": "Inverness", "lat": 57.477773, "lon": -4.224721},
    {"name": "Kingston upon Hull", "lat": 53.7674909, "lon": -0.3273025},
    {"name": "Lancaster", "lat": 54.0449072, "lon": -2.7993467},
    {"name": "Leeds", "lat": 53.8007554, "lon": -1.5490774},
    {"name": "Leicester", "lat": 52.6368778, "lon": -1.1397592},
    {"name": "Lichfield", "lat": 52.681602, "lon": -1.831672},
    {"name": "Lincoln", "lat": 53.2331, "lon": -0.5392},
    {"name": "Lisburn", "lat": 54.516246, "lon": -6.0580106},
    {"name": "Liverpool", "lat": 53.4083714, "lon": -2.9915726},
    {"name": "Manchester", "lat": 53.4807593, "lon": -2.2426305},
    {"name": "Milton Keynes", "lat": 52.0406224, "lon": -0.7594171},
    {"name": "Newcastle", "lat": 54.978252, "lon": -1.61778},
    {"name": "Newport", "lat": 51.5842, "lon": -2.9977},
    {"name": "Newry", "lat": 54.1751024, "lon": -6.3402299},
    {"name": "Norwich", "lat": 52.6292567, "lon": 1.2978802},
    {"name": "Nottingham", "lat": 52.9540223, "lon": -1.1549892},
    {"name": "Oxford", "lat": 51.7520209, "lon": -1.2577263},
    {"name": "Perth", "lat": 56.3969, "lon": -3.4370},
    {"name": "Peterborough", "lat": 52.5703169, "lon": -0.2407997},
    {"name": "Plymouth", "lat": 50.3754565, "lon": -4.1426565},
    {"name": "Portsmouth", "lat": 50.8197675, "lon": -1.0879769},
    {"name": "Preston", "lat": 53.7591576, "lon": -2.7046445},
    {"name": "Ripon", "lat": 54.1361346, "lon": -1.5237756},
    {"name": "Salford", "lat": 53.4872927, "lon": -2.2900071},
    {"name": "Salisbury", "lat": 51.068785, "lon": -1.794472},
    {"name": "Sheffield", "lat": 53.381129, "lon": -1.470085},
    {"name": "Southampton", "lat": 50.9105468, "lon": -1.4049018},
    {"name": "Southend-on-Sea", "lat": 51.5459269, "lon": 0.7077123},
    {"name": "St Albans", "lat": 51.7508784, "lon": -0.3397723},
    {"name": "St Asaph", "lat": 53.2577125, "lon": -3.4418762},
    {"name": "St Davids", "lat": 51.881227, "lon": -5.265995},
    {"name": "Stirling", "lat": 56.1165227, "lon": -3.9369029},
    {"name": "Stoke-on-Trent", "lat": 53.0033369, "lon": -2.1827408},
    {"name": "Sunderland", "lat": 54.9044161, "lon": -1.3811746},
    {"name": "Swansea", "lat": 51.62144, "lon": -3.943646},
    {"name": "Truro", "lat": 50.263195, "lon": -5.051041},
    {"name": "Wakefield", "lat": 53.6848162, "lon": -1.5038596},
    {"name": "Wells", "lat": 51.2104862, "lon": -2.6467584},
    {"name": "Winchester", "lat": 51.059771, "lon": -1.310142},
    {"name": "Wolverhampton", "lat": 52.5868159, "lon": -2.1256587},
    {"name": "Worcester", "lat": 52.1941389, "lon": -2.2190523},
    {"name": "Wrexham", "lat": 53.04304, "lon": -2.992494},
    {"name": "York", "lat": 53.9614205, "lon": -1.0739108}
]

data = []

for city in cities:
    print(f"\nFetching crime data for {city['name']} over 12 months...")
    total_crimes = 0
    all_crime_types = []

    for month in months:
        url = f"https://data.police.uk/api/crimes-at-location?date={month}&lat={city['lat']}&lng={city['lon']}"
        try:
            response = requests.get(url, timeout=15)
            if response.status_code == 200:
                crimes = response.json()
                total_crimes += len(crimes)
                all_crime_types.extend([crime["category"] for crime in crimes])
            else:
                print(f"⚠️ No data for {city['name']} ({month}) - HTTP {response.status_code}")
        except Exception as e:
            print(f"Error fetching {city['name']} for {month}: {e}")
        time.sleep(1.2)  # Avoid API rate limits

    # Count each type of crime across all months
    crime_count_by_type = Counter(all_crime_types)

    # Build row
    row = {"City": city["name"], "Total_Crimes": total_crimes}
    for ctype, count in crime_count_by_type.items():
        row[ctype] = count

    data.append(row)

# Convert to DataFrame
df = pd.DataFrame(data)
df.fillna(0, inplace=True)

# Save to CSV
df.to_csv("uk_crime_yearly.csv", index=False)
print("\n✅ Crime data saved to uk_crime_yearly.csv")



Fetching crime data for Aberdeen over 12 months...

Fetching crime data for Armagh over 12 months...

Fetching crime data for Bangor(NI) over 12 months...

Fetching crime data for Bangor(W) over 12 months...

Fetching crime data for Bath over 12 months...

Fetching crime data for Belfast over 12 months...

Fetching crime data for Birmingham over 12 months...

Fetching crime data for Bradford over 12 months...

Fetching crime data for Brighton and Hove over 12 months...

Fetching crime data for Bristol over 12 months...

Fetching crime data for Cambridge over 12 months...

Fetching crime data for Canterbury over 12 months...

Fetching crime data for Cardiff over 12 months...

Fetching crime data for Carlisle over 12 months...

Fetching crime data for Chelmsford over 12 months...

Fetching crime data for Chester over 12 months...

Fetching crime data for Chichester over 12 months...

Fetching crime data for City of London over 12 months...

Fetching crime data for City of Westminster ov

In [ ]:
import pandas as pd
import requests
from io import BytesIO
from bs4 import BeautifulSoup

# Download file
url = "https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/housing/datasets/privaterentalmarketsummarystatisticsinengland/october2021toseptember2022/privaterentalmarketstatistics221214.xlsx"
print("Downloading ONS data from:", url)

response = requests.get(url)
response.raise_for_status()

# Load local authority sheet (city-level data)
xls = pd.ExcelFile(BytesIO(response.content))
print("Available sheets:", xls.sheet_names)

# Table 2.15 is typically the local authority sheet — may vary by version
target_sheet = [s for s in xls.sheet_names if "2.3" in s or "2_3" in s]
if not target_sheet:
    target_sheet = ["Table2.3"]  # fallback name
target_sheet = target_sheet[0]

df = pd.read_excel(xls, sheet_name=target_sheet, skiprows=7)
print(df.columns.tolist())
df.head(10)


# # Step 3: Clean ONS data
# df = df.rename(columns={
#     "Local authority name": "City",
#     "Median": "Median Rent (£)",
#     "Property Type": "Property Type"
# })

# df = df[["City", "Property Type", "Median Rent (£)"]].dropna()

# # Step 4: Pivot into city-wise structure
# property_types = ["Room", "1 bed", "2 bed", "3 bed", "4 bed"]
# df = df[df["Property Type"].isin(property_types)]
# df_pivot = df.pivot_table(index="City", columns="Property Type", values="Median Rent (£)", aggfunc="mean").reset_index()

# # Step 5: Save clean data
# df_pivot.to_csv("ons_rent_data.csv", index=False)
# print("✅ Saved as ons_rent_data.csv")


Available sheets: ['Cover sheet', 'Contents', 'Table 1.1', 'Table 1.2', 'Table 1.3', 'Table 1.4', 'Table 1.5', 'Table 1.6', 'Table 1.7', 'Table2.1', 'Table2.2', 'Table2.3', 'Table2.4', 'Table2.5', 'Table2.6', 'Table2.7', 'Contact']
['Unnamed: 0', 'NA', 'E92000001', 'ENGLAND', 97860, 791, 550, 715, 925]


,Unnamed: 0,NA,E92000001,ENGLAND,97860,791,550,715,925
0,NaN,NaN,E12000001,NORTH EAST,2190.0,468.0,375.0,430.0,525.0
1,NaN,1355.0,E06000047,County Durham UA,270.0,428.0,325.0,380.0,450.0
2,NaN,1350.0,E06000005,Darlington UA,290.0,400.0,347.0,385.0,450.0
3,NaN,724.0,E06000001,Hartlepool UA,110.0,390.0,360.0,395.0,420.0
4,NaN,734.0,E06000002,Middlesbrough UA,70.0,440.0,404.0,430.0,455.0
5,NaN,2935.0,E06000057,Northumberland UA,210.0,411.0,350.0,400.0,455.0
6,NaN,728.0,E06000003,Redcar and Cleveland UA,100.0,409.0,375.0,412.0,429.0
7,NaN,738.0,E06000004,Stockton-on-Tees UA,160.0,437.0,390.0,430.0,475.0
8,NaN,NaN,E11000007,Tyne and Wear (Met County),980.0,534.0,425.0,500.0,625.0
9,NaN,4505.0,E08000037,Gateshead,120.0,495.0,400.0,475.0,575.0


In [ ]:
import pandas as pd

# --- Step 1: Load data ---
df = pd.read_csv("/content/Rent_Sheet1.csv")  # your file path

# --- Step 2: Columns to normalize ---
rent_cols = ["1_BedRoom", "2_BedRoom", "3_BedRoom", "4+_BedRoom"]

# --- Step 3: Compute min-max normalization for each rent type ---
for col in rent_cols:
    min_val = df[col].min()
    max_val = df[col].max()
    df[f"{col}_Score"] = 1 - (df[col] - min_val) / (max_val - min_val)
    df[f"{col}_Score"] = df[f"{col}_Score"].round(3)

# --- Step 4: Weighted overall Rent Score ---
weights = {
    "1_BedRoom_Score": 0.45,
    "2_BedRoom_Score": 0.35,
    "3_BedRoom_Score": 0.15,
    "4+_BedRoom_Score": 0.05
}

df["Overall_Rent_Score"] = sum(df[col] * w for col, w in weights.items())
df["Overall_Rent_Score"] = df["Overall_Rent_Score"].round(3)

# --- Step 5: Save and inspect ---
df.to_csv("rent_scores.csv", index=False)
print("✅ Rent scores saved to rent_scores_detailed.csv")

print("\nTop 5 most affordable cities (Overall):")
print(df.sort_values("Overall_Rent_Score", ascending=False).head())

print("\nTop 5 most expensive cities (Overall):")
print(df.sort_values("Overall_Rent_Score").head())


✅ Rent scores saved to rent_scores_detailed.csv

Top 5 most affordable cities (Overall):
                  City  1_BedRoom  2_BedRoom  3_BedRoom  4+_BedRoom  Points  \
26              Durham        421        486        574         906     NaN   
2           Bangor(NI)        432        500        588         600     NaN   
34  Kingston upon Hull        437        508        593         796     NaN   
13            Carlisle        427        529        658         935     NaN   
65      Stoke-on-Trent        452        530        659         926     NaN   

    1_BedRoom_Score  2_BedRoom_Score  3_BedRoom_Score  4+_BedRoom_Score  \
26            1.000            1.000            1.000             0.965   
2             0.994            0.995            0.997             1.000   
34            0.992            0.992            0.996             0.978   
13            0.997            0.985            0.981             0.962   
65            0.984            0.985            0.980        

In [ ]:
import pandas as pd

# --- Step 1: Load crime dataset ---
df = pd.read_csv("/content/uk_crime_yearly.csv")  # your file path

# --- Step 2: Columns to normalize ---
crime_cols = ["Avg_Crimes_per_km2_yearly", "Mild", "Serious", "Dangerous"]

# --- Step 3: Normalize each (inverse since lower = better) ---
for col in crime_cols:
    min_val = df[col].min()
    max_val = df[col].max()
    df[f"{col}_Score"] = 1 - (df[col] - min_val) / (max_val - min_val)
    df[f"{col}_Score"] = df[f"{col}_Score"].round(3)

# --- Step 4: Weighted crime safety score ---
weights = {
    "Mild_Score": 0.1,
    "Serious_Score": 0.3,
    "Dangerous_Score": 0.6
}

df["Overall_Crime_Safety_Score"] = sum(df[col] * w for col, w in weights.items())
df["Overall_Crime_Safety_Score"] = df["Overall_Crime_Safety_Score"].round(3)

# --- Step 5: Combine with average crimes for context ---
df["Overall_Safety_Score"] = (df["Overall_Crime_Safety_Score"] * 0.7 +
                              df["Avg_Crimes_per_km2_yearly_Score"] * 0.3)
df["Overall_Safety_Score"] = df["Overall_Safety_Score"].round(3)

# --- Step 6: Save results ---
df.to_csv("crime_scores.csv", index=False)
print("✅ Crime scores saved to crime_scores_detailed.csv")

# --- Step 7: Preview ---
print("\nTop 5 Safest Cities:")
print(df.sort_values("Overall_Safety_Score", ascending=False).head())

print("\nTop 5 Most Risky Cities:")
print(df.sort_values("Overall_Safety_Score").head())


✅ Crime scores saved to crime_scores_detailed.csv

Top 5 Safest Cities:
          City  Avg_Crimes_per_km2_yearly  criminal-damage-arson  drugs  \
56     Salford                          0                      0      0   
2   Bangor(NI)                          1                      1      0   
39     Lincoln                          2                      0      0   
63   St Davids                          3                      0      0   
40     Lisburn                          1                      0      0   

    other-theft  possession-of-weapons  shoplifting  violent-crime  \
56            0                      0            0              0   
2             0                      0            0              0   
39            1                      0            0              0   
63            1                      1            0              0   
40            0                      0            0              0   

    other-crime  anti-social-behaviour  ...  Mild  Serio

In [ ]:
import pandas as pd

# Load your data (example path)
df = pd.read_csv("/content/uk_uni.csv")

# Columns to normalize
cols = ["Universities", "Top 100", "Top 500", "Top 2000"]

# Normalize each (higher = better)
for col in cols:
    min_val = df[col].min()
    max_val = df[col].max()
    df[f"{col}_Score"] = (df[col] - min_val) / (max_val - min_val)
    df[f"{col}_Score"] = df[f"{col}_Score"].round(3)

# Assign weights
weights = {
    "Universities_Score": 0.30,
    "Top 100_Score": 0.35,
    "Top 500_Score": 0.25,
    "Top 2000_Score": 0.10
}

# Weighted final score
df["University_Quality_Score"] = sum(df[col] * w for col, w in weights.items())
df["University_Quality_Score"] = df["University_Quality_Score"].round(3)

# Save results
df.to_csv("uni_scores.csv", index=False)
print("✅ University scores saved to university_scores.csv")

# Optional: Show top 5 cities
print("\nTop 5 university cities:")
print(df.sort_values("University_Quality_Score", ascending=False).head())


✅ University scores saved to university_scores.csv

Top 5 university cities:
          City  Universities  Top 100  Top 500  Top 2000  Points  \
10   Cambridge           113        1        3       109     NaN   
49      Oxford            82        1        3        78     NaN   
12     Cardiff            55        1        4        50     NaN   
6   Birmingham            36        1        5        30     NaN   
5      Belfast            14        1        5         8     NaN   

    Universities_Score  Top 100_Score  Top 500_Score  Top 2000_Score  \
10               1.000            1.0            0.6           1.000   
49               0.726            1.0            0.6           0.716   
12               0.487            1.0            0.8           0.459   
6                0.319            1.0            1.0           0.275   
5                0.124            1.0            1.0           0.073   

    University_Quality_Score  
10                     0.900  
49                 

In [ ]:
import pandas as pd

# === Step 1: Load your healthcare data ===
# Replace this path with your actual CSV file name
df = pd.read_csv("healthcare.csv")

# Expected columns:
# City, Major Hospitals, Major Pharmacies, Population
# Optional: Hospital Density, Pharmacy Density (if not already there)

# === Step 2: Calculate densities if not already present ===
if "Hospital Density" not in df.columns:
    df["Hospital Density"] = df["Population"] / df["Major Hospitals"]

if "Pharmacy Density" not in df.columns:
    df["Pharmacy Density"] = df["Population"] / df["Major Pharmacies"]

# === Step 3: Normalize (inverse — lower density = better access) ===
for col in ["Hospital Density", "Pharmacy Density"]:
    min_val = df[col].min()
    max_val = df[col].max()
    df[f"{col} Score"] = 1 - (df[col] - min_val) / (max_val - min_val)
    df[f"{col} Score"] = df[f"{col} Score"].clip(0, 1).round(3)

# === Step 4: Combine weighted scores ===
# Hospitals weigh more since they reflect core healthcare availability
df["Healthcare Score"] = (
    0.7 * df["Hospital Density Score"] +
    0.3 * df["Pharmacy Density Score"]
).round(3)

# === Step 5: Save to new CSV ===
output_file = "healthcare_scores.csv"
df.to_csv(output_file, index=False)

print(f"✅ Healthcare scores saved to {output_file}")
print(df[["City", "Hospital Density Score", "Pharmacy Density Score", "Healthcare Score"]].head())


✅ Healthcare scores saved to healthcare_scores.csv
          City  Hospital Density Score  Pharmacy Density Score  \
0     Aberdeen                   0.868                   0.868   
1       Armagh                   0.984                   0.987   
2  Bangor (NI)                   0.703                   0.933   
3   Bangor (W)                   0.917                   0.969   
4         Bath                   0.769                   0.849   

   Healthcare Score  
0             0.868  
1             0.985  
2             0.772  
3             0.933  
4             0.793  


In [ ]:
import pandas as pd

# === Step 1: Load air quality data ===
# Expected columns: City, AQI
df = pd.read_csv("air_quality.csv")

# === Step 2: Normalize AQI ===
# Lower AQI = better air → higher score
min_aqi = df["AQI"].min()
max_aqi = df["AQI"].max()

df["Air_Quality_Score"] = 1 - (df["AQI"] - min_aqi) / (max_aqi - min_aqi)
df["Air_Quality_Score"] = df["Air_Quality_Score"].clip(0, 1).round(3)

# === Step 3: Save results ===
output_file = "air_quality_scores.csv"
df.to_csv(output_file, index=False)

print(f"✅ Air quality scores saved to {output_file}")
print(df.head())


✅ Air quality scores saved to air_quality_scores.csv
          City  AQI  Points  Air_Quality_Score
0     Aberdeen   27     NaN              0.866
1       Armagh   11     NaN              0.948
2  Bangor (NI)   21     NaN              0.897
3   Bangor (W)   24     NaN              0.881
4         Bath    9     NaN              0.959


In [ ]:
import pandas as pd
import numpy as np

# === Step 1: Load data ===
# Expected columns: City, Population, Area (km²), Density
df = pd.read_csv("Population.csv")

# === Step 2: Define ideal parameters ===
ideal_density = 1500   # ideal population density
sigma = 800            # spread parameter

# === Step 3: Apply bell-shaped scoring ===
df["Population_Score"] = np.exp(-((df["Density"] - ideal_density)**2 / (2 * sigma**2)))
df["Population_Score"] = df["Population_Score"].round(3)

# === Step 4: Save results ===
df.to_csv("population_scores.csv", index=False)

print("✅ Population density scores saved to population_scores.csv")
print(df.head())


✅ Population density scores saved to population_scores.csv
         City Population  Area (km²)      Density  Population_Score
0    Aberdeen     217260      185.07  1173.934187             0.920
1      Armagh      16310       28.00   582.500000             0.518
2  Bangor(NI)      61011       35.00  1743.171429             0.955
3   Bangor(W)     16,990        6.50  2613.846154             0.379
4        Bath      94782      115.00   824.191304             0.700
